In [44]:
import numpy as np
import pandas as pd
from itertools import product
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# Load the dataset
path='/Volumes/2TB Ext/BrainBaseMHQ/mhm_data_2024-01-18_17-04-32.csv'
path2 = '/Users/jerzybala/Desktop/Simulation Experiments Aug-Sep 2023/PROCESSED_FOOD_Dec2023_raw.csv'

data = pd.read_csv(path2)
data.rename_axis('row_id', inplace=True, axis=1)

# Create the "MHQ_Sign" column
data['MHQ_Sign'] = data['Overall MHQ'].apply(lambda x: 1 if x > 0 else 0)
print(data.shape)

data.columns.tolist()

(401682, 238)


['Unnamed: 0',
 'Language',
 'Start Date (UTC)',
 'Submit Date (UTC)',
 'Time to completion',
 'Overall MHQ',
 'Cognition Dimension',
 'Adaptability & Resilence Dimension',
 'Drive & Motivation Dimension',
 'Mood & Outlook Dimension',
 'Social & Self Dimension',
 'Mind Body Connection Dimension',
 'Understanding of Assessment',
 'Adaptability to Change',
 'Self Worth & Confidence',
 'Creativity & Problem Solving',
 'Drive & Motivation',
 'Stability & Calmness',
 'Sleep Quality',
 'Self Control & Impulsivity',
 'Ability to Learn',
 'Coordination',
 'Relationships with others',
 'Emotional Resilience',
 'Planning & Organisation',
 'Physical Intimacy',
 'Speech & Language',
 'Memory',
 'Social interactions & Cooperation',
 'Decision-making & Risk-taking',
 'Curiosity, Interest & Enthusiasm',
 'Energy Level',
 'Emotional Control',
 'Focus & Concentration',
 'Appetite Regulation',
 'Empathy',
 'Sensory Sensitivity',
 'Self-Image',
 'Outlook & Optimism',
 'Selective Attention',
 'Restlessnes

In [45]:
#print(data['Processed food in diet'].value_counts())
print(data['Processed food in diet'].unique().tolist())


nan_count = data['Processed food in diet'].isna().sum()
blank_count = (data['Processed food in diet'] == '').sum()

total_empty = nan_count + blank_count
print("Number of NaN values:", nan_count)
print("Number of blank string values:", blank_count)
print("Total number of empty values:", total_empty)
print("Remaining after considering food_values:", data.shape)

# # Remove rows where 'Processed food in diet' is either NaN or a blank string
# data.shape
# data = data[~data['Processed food in diet'].isna() & (data['Processed food in diet'] != '')]
# data.shape

# print("Remaining rows after removal:", len(data))

      
# food_values=['A few times a week', 'Rarely/Never', 'A few times a month',
#        'Several times a day', 'Rarely/never', 'Once a day',
#        'Many times in a day', 'At least once a day',
#        'A few times in a day', 'Several days a week']

# data = data[data['Processed food in diet'].isin(food_values)]
# print("Remaining after considering food_values:", len(data))

# data.columns.tolist()

['A few times a week', 'Rarely/Never', 'A few times a month', 'Several times a day', 'Rarely/never', 'Once a day', 'Many times in a day', 'At least once a day', 'A few times in a day', 'Several days a week']
Number of NaN values: 0
Number of blank string values: 0
Total number of empty values: 0
Remaining after considering food_values: (401682, 238)


In [46]:
# ONE HOT ENCODING

# Values for different categories
category_values = {
    'Adult traumas': [
        'Sudden or premature death of a loved one',  
        'Divorce/separation  or family breakup',
        'Extreme poverty leading to homelessness and/or hunger.',
        'Divorce/separation  or family breakup',
        'Forced family control over major life decisions (e.g. marriage)',
        'Prolonged sexual abuse| or severe sexual assault.',
        'Displacement from your home due to political| environmental or economic reasons',
        'Loss of your job or livelihood leading to an inability to make ends meet.',
        'Cyberbullying or online abuse',
        'Threatening| coercive or controlling behavior by another person',
        'Caring for a child or partner with a major chronic disability or illness',
        'I did not experience any of the above',
        'Involvement or close witness to a war',
        'Life threatening or debilitating injury or illness.',
        'Suffered a loss in a major fire| flood| earthquake| or natural disaster',
        'None of the above AT'
        
    ],
    'Type of Medical Condition': [
        'Type II Diabetes', 'Fibromyalgia', 'Liver disease/Cirrhosis', 
        'Hypertension', 'Irritable Bowel Syndrome', 'Heart disease', 
        "Inflammatory Bowel Disease / Crohn's disease", 'Arthritis', 
        'Psoriasis', 'Asthma', 'Migraines', 'Traumatic Brain Injury', 
        'Osteoporosis', 'Sleep apnea', 'Hipotiroidismo', 'Neuropathy', 
        'Cancer', 'Chronic Obstructive Pulmonary Disease (COPD)', 'HIV /AIDS', 
        'Liver disease/Cirrhosis', 'Kidney Disease', 'Heart disease', 
        'Rheumatoid', 'Polycystic ovaries', 'Prefer not to say', 'Narcolepsy', 
        'Chronic fatigue syndrome', 'Back problem', 'Epilepsy', 'Multiple sclerosis', 
        'Escoliosis', 'Stroke', 'Kidney Disease', 'Herpes', 'Type 1 Diabetes'
    ],
    'Childhood traumas': [
        'Sudden or premature death of a parent or sibling',
        'Prolonged emotional or psychological abuse or neglect from parent/caregiver',
        'Prolonged physical abuse| or severe physical assault CT',
        'Physical violence in the home between family members',
        'Prolonged or sustained bullying in person from peers',
        'Involvement or close witness to a war',
        'Parental Divorce or family breakup',
        'Lived with a parent/caregiver who was an alcoholic or who regularly used street drugs',
        'Threatening| coercive or controlling behavior by another person CT',
        'I did not experience any of the above during my childhood',
        'Suffered a loss in a major fire| flood| earthquake| or natural disaster CT',
        'Displacement from your home due to political| environmental or economic reasons CT',
        'Life threatening or debilitating injury or illness CT',
        'Forced family control over major life decisions CT',
        'None of the above CT'
        
    ],
    'Substance Use': [
        'Tobacco products',
        'Alcoholic beverages',
        'Cannabis',
        'Vaping products',
        'Sedatives or Sleeping Pills',
        'Amphetamine type stimulants (e.g. speed| diet pills| ecstasy| etc.)',
        'Opioids',
        'Melatonina',
        'Cigarrillo',
        'None of the above'
    ]
}

df1=data

for column, values in category_values.items():
    for c in values:
        df1[c] = np.where(
            df1[column].str.contains(c) & ~df1[column].str.strip().eq(''),
            1, 0
        )

data=df1
#data=data.drop(['Adult traumas', 'Childhood traumas', 'Type of Medical Condition', 'Substance Use'], axis=1)
data.to_csv('/Users/jerzybala/Desktop/check.csv', index = True)

data.shape

(401682, 238)

In [ ]:
# ONE HOT ENCODING

# Values for different categories
category_values = {
   
    'Childhood trauma': [
        'Sudden or premature death of a parent or sibling',
        'Prolonged emotional or psychological abuse or neglect from parent/caregiver',
        'Prolonged physical abuse| or severe physical assault CT',
        'Physical violence in the home between family members',
        'Prolonged or sustained bullying in person from peers',
        'Involvement or close witness to a war',
        'Parental Divorce or family breakup',
        'Lived with a parent/caregiver who was an alcoholic or who regularly used street drugs',
        'Threatening| coercive or controlling behavior by another person CT',
        'I did not experience any of the above during my childhood',
        'Suffered a loss in a major fire| flood| earthquake| or natural disaster CT',
        'Displacement from your home due to political| environmental or economic reasons CT',
        'Life threatening or debilitating injury or illness CT',
        'Forced family control over major life decisions CT',
        'None of the above CT'
        
    ],
    'Substance Use': [
        'Tobacco products',
        'Alcoholic beverages',
        'Cannabis',
        'Vaping products',
        'Sedatives or Sleeping Pills',
        'Amphetamine type stimulants (e.g. speed| diet pills| ecstasy| etc.)',
        'Opioids',
        'Melatonina',
        'Cigarrillo',
        'None of the above'
    ]
}

df1=data

for column, values in category_values.items():
    for c in values:
        df1[c] = np.where(
            df1[column].str.contains(c) & ~df1[column].str.strip().eq(''),
            1, 0
        )

data=df1
#data=data.drop(['Adult traumas', 'Childhood traumas', 'Type of Medical Condition', 'Substance Use'], axis=1)
data.to_csv('/Users/jerzybala/Desktop/check.csv', index = True)

data.shape

In [20]:
data = data[data['Age'] == '18-24']
print(data.shape)


(187997, 238)


In [21]:
data.columns.tolist()


['Unnamed: 0',
 'Language',
 'Start Date (UTC)',
 'Submit Date (UTC)',
 'Time to completion',
 'Overall MHQ',
 'Cognition Dimension',
 'Adaptability & Resilence Dimension',
 'Drive & Motivation Dimension',
 'Mood & Outlook Dimension',
 'Social & Self Dimension',
 'Mind Body Connection Dimension',
 'Understanding of Assessment',
 'Adaptability to Change',
 'Self Worth & Confidence',
 'Creativity & Problem Solving',
 'Drive & Motivation',
 'Stability & Calmness',
 'Sleep Quality',
 'Self Control & Impulsivity',
 'Ability to Learn',
 'Coordination',
 'Relationships with others',
 'Emotional Resilience',
 'Planning & Organisation',
 'Physical Intimacy',
 'Speech & Language',
 'Memory',
 'Social interactions & Cooperation',
 'Decision-making & Risk-taking',
 'Curiosity, Interest & Enthusiasm',
 'Energy Level',
 'Emotional Control',
 'Focus & Concentration',
 'Appetite Regulation',
 'Empathy',
 'Sensory Sensitivity',
 'Self-Image',
 'Outlook & Optimism',
 'Selective Attention',
 'Restlessnes

In [22]:
# features for causal modeling

causal_features1 = [
 'Self Worth & Confidence',
 'Creativity & Problem Solving',
 'Drive & Motivation',
 'Stability & Calmness',
 'Sleep Quality',
 'Self Control & Impulsivity',
 'Ability to Learn',
 'Coordination',
 'Relationships with others',
 'Emotional Resilience',
 'Planning & Organisation',
 'Physical Intimacy',
 'Speech & Language',
 'Memory',
 'Social interactions & Cooperation',
 'Decision-making & Risk-taking',
 'Curiosity, Interest & Enthusiasm',
 'Energy Level',
 'Emotional Control',
 'Focus & Concentration',
 'Appetite Regulation',
 'Empathy',
 'Sensory Sensitivity',
 'Self-Image',
 'Outlook & Optimism',
 'Selective Attention',
 'Restlessness & Hyperactivity',
 'Fear & Anxiety',
 'Susceptibility to Infections',
 'Aggression Towards Others',
 'Avoidance & Withdrawal',
 'Unwanted, Strange or Obsessive Thoughts',
 'Mood Swings',
 'Sense of being detached from reality',
 'Nightmares',
 'Addictions',
 'Anger & Irritability',
 'Suicidal Thoughts or Intentions',
 'Experience of Pain',
 'Guilt & Blame',
 'ARCHIVED: Options that best describe your feelings of Guilt and Blame',
 'Hallucinations',
 'Traumatic Flashbacks',
 'Repetitive or Compulsive Actions',
 'Feelings of Sadness, Distress or Hopelessness',
 'Physical Health Issues',
 'Confusion or Slowed Thinking',



    
 'Frequency of getting a good nights sleep',
 'Frequency of doing exercise',
 'Processed food in diet',
 'Frequency of Socializing',
 

 'Sudden or premature death of a parent or sibling',
 'Prolonged emotional or psychological abuse or neglect from parent/caregiver',
 'Prolonged physical abuse| or severe physical assault CT',
 'Physical violence in the home between family members',
 'Prolonged or sustained bullying in person from peers',
 'Parental Divorce or family breakup',
 'Lived with a parent/caregiver who was an alcoholic or who regularly used street drugs',
 'Threatening| coercive or controlling behavior by another person CT',
 'I did not experience any of the above during my childhood',
 'Suffered a loss in a major fire| flood| earthquake| or natural disaster CT',
 'Displacement from your home due to political| environmental or economic reasons CT',
 'Life threatening or debilitating injury or illness CT',
 'Forced family control over major life decisions CT',

    
 'Cannabis',
 'Vaping products']



In [23]:
data = data[causal_features1]


In [24]:
data.to_csv('/Users/jerzybala/Desktop/causal1.csv', index=False)


In [8]:
#data.columns.to_list()
for column in data.columns:
    print(f"'{column}'", end=",")


'Language','Start Date (UTC)','Submit Date (UTC)','Time to completion','Overall MHQ','Cognition Dimension','Adaptability & Resilence Dimension','Drive & Motivation Dimension','Mood & Outlook Dimension','Social & Self Dimension','Mind Body Connection Dimension','Understanding of Assessment','Adaptability to Change','Self Worth & Confidence','Creativity & Problem Solving','Drive & Motivation','Stability & Calmness','Sleep Quality','Self Control & Impulsivity','Ability to Learn','Coordination','Relationships with others','Emotional Resilience','Planning & Organisation','Physical Intimacy','Speech & Language','Memory','Social interactions & Cooperation','Decision-making & Risk-taking','Curiosity, Interest & Enthusiasm','Energy Level','Emotional Control','Focus & Concentration','Appetite Regulation','Empathy','Sensory Sensitivity','Self-Image','Outlook & Optimism','Selective Attention','Restlessness & Hyperactivity','Fear & Anxiety','Susceptibility to Infections','Aggression Towards Others'

In [9]:
data=data.drop(['Language','Start Date (UTC)','Submit Date (UTC)','Time to completion','Cognition Dimension','Adaptability & Resilence Dimension','Drive & Motivation Dimension','Mood & Outlook Dimension','Social & Self Dimension','Mind Body Connection Dimension','Understanding of Assessment','Adaptability to Change','Self Worth & Confidence','Creativity & Problem Solving','Drive & Motivation','Stability & Calmness','Sleep Quality','Self Control & Impulsivity','Ability to Learn','Coordination','Relationships with others','Emotional Resilience','Planning & Organisation','Physical Intimacy','Speech & Language','Memory','Social interactions & Cooperation','Decision-making & Risk-taking','Curiosity, Interest & Enthusiasm','Energy Level','Emotional Control','Focus & Concentration','Appetite Regulation','Empathy','Sensory Sensitivity','Self-Image','Outlook & Optimism','Selective Attention','Restlessness & Hyperactivity','Fear & Anxiety','Susceptibility to Infections','Aggression Towards Others','Avoidance & Withdrawal','Unwanted, Strange or Obsessive Thoughts','ARCHIVED: Primary nature of strange, unwanted or obsessive thoughts','Mood Swings','Sense of being detached from reality','Nightmares','Addictions','Anger & Irritability','Suicidal Thoughts or Intentions','Experience of Pain','Guilt & Blame','ARCHIVED: Options that best describe your feelings of Guilt and Blame','Hallucinations','Traumatic Flashbacks','Repetitive or Compulsive Actions','Feelings of Sadness, Distress or Hopelessness','Physical Health Issues','Confusion or Slowed Thinking'], axis=1)

In [10]:
data=data.drop(['ARCHIVED: Friendship (type of interaction)','Friendship (help you out)','ARCHIVED:riendship (confide in)','ARCHIVED: Nature of Household Growing up','Description of Household Growing up','ARCHIVED: Parental/caregiver support','ARCHIVED: Proximity of Adult Family','Relationship with Adult Family','ARCHIVED: Individualism vs collectivism','ARCHIVED: PHQ-9 - Interest','ARCHIVED: PHQ-9 - Depressed','ARCHIVED: PHQ-9 - Sleep','ARCHIVED: PHQ-9 - Energy','ARCHIVED: PHQ-9 - Failure','ARCHIVED: PHQ-9 - Appetite','ARCHIVED: PHQ-9 - Concentration','ARCHIVED: PHQ-9 - Slow movements','ARCHIVED: PHQ-9 - Self harm','ARCHIVED: GAD-7 - Feeling nervous','ARCHIVED: GAD-7 - Worrying','ARCHIVED: GAD-7 - Self-control','ARCHIVED: GAD-7 - Trouble relaxing','ARCHIVED: GAD-7 - Restless','ARCHIVED: GAD-7 - Annoyed or irritable','ARCHIVED: GAD-7 - Feeling afraid','ARCHIVED: GAD-7 - Impact of the problems','ARCHIVED: Life Satisfaction','ARCHIVED: Time of day','ARCHIVED: Mood','ARCHIVED: Alertness','ARCHIVED: Hours Sleep Previous Night','ARCHIVED: Time Since Last Meal','ARCHIVED: Physical complaints','ARCHIVED: Pregnant','ARCHIVED: Covid: health & social adversities','ARCHIVED: Covid: financial adversities','ARCHIVED: Depression Benchmarking (frequency)','ARCHIVED: Depression Benchmarking (severity)','Admin Use 1','Admin Use 2','Admin Use 3','Repeat identifier'], axis=1)


In [11]:
data=data.drop(['ARCHIVED: Gender', 'ARCHIVED: Type of Treatment','Effectiveness of therapy treatment','Effectiveness of medication treatment','Effectiveness of brain stimulation treatment','Effectiveness of neurofeedback treatment','Diagnosed mental health disorders','ARCHIVED: Life Trauma (pre-2022)','ARCHIVED: Childhood Friends','ARCHIVED: Proximity of Close Friends','ARCHIVED: Job Role','Household Income','ARCHIVED: Productivity Benchmarking (unable to work)','ARCHIVED: Productivity Benchmarking (unproductive work)', 'Presence/Absence of Diagnosed Medical Disorder', 'Mental Health Treatment Status', 'Not seeking help reasons', 'Type of Treatment (revised 2023)'], axis=1)


In [9]:
from tabulate import tabulate

# Create a list to store the data for tabulation
table_data = []

# Populate the table_data list
for column in data.columns:
    unique_count = data[column].nunique()
    table_data.append([column, unique_count])
    
#table_data.sort(key=lambda x: x[0])
#table_data.sort(key=lambda x: x[0], reverse=True)

# Tabulate the data and print as a table
table_headers = ['Column Name', 'Unique Values']
table = tabulate(table_data, headers=table_headers, tablefmt='grid')
print(table)


+---------------------------------------------------------------------------------------+-----------------+
| Column Name                                                                           |   Unique Values |
+=======================================================================================+=================+
| Language                                                                              |              13 |
+---------------------------------------------------------------------------------------+-----------------+
| Start Date (UTC)                                                                      |         1180767 |
+---------------------------------------------------------------------------------------+-----------------+
| Submit Date (UTC)                                                                     |         1180900 |
+---------------------------------------------------------------------------------------+-----------------+
| Time to completion        

In [13]:
data=data.drop(['Age of smartphone access', 'Smartphone allowed in school', 'Smartphone use in lessons', 
                'Internet restriction as a teenager', 'Age of first social media account', 
                'Frequency of social media posting', 'Tablet ownership', 'Smartphone ownership.1',
                'Friends/classmates smarphone ownership', 'Age of smartphone usage during school hours', 
                'Smartphone usage during class hours', 'Smartphone usage during break', 
                'Age of laptop usage required by school', 'Laptop usage for non-learning activities'], axis=1)


In [13]:
data=data.drop(['Ethnicity', 'State'], axis=1)


In [14]:
data.reset_index(inplace=True)
data.rename(columns={'index': 'row_id'}, inplace=True)
# Replace spaces with underscores in column names
data.columns = data.columns.str.replace(' ', '_')


In [16]:
from tabulate import tabulate

# Create a list to store the data for tabulation
table_data = []

# Populate the table_data list
for column in data.columns:
    unique_count = data[column].nunique()
    table_data.append([column, unique_count])
    
#table_data.sort(key=lambda x: x[0])
#table_data.sort(key=lambda x: x[0], reverse=True)

# Tabulate the data and print as a table
table_headers = ['Column Name', 'Unique Values']
table = tabulate(table_data, headers=table_headers, tablefmt='grid')
print(table)


+---------------------------------------------------------------------------------------+-----------------+
| Column Name                                                                           |   Unique Values |
+=======================================================================================+=================+
| row_id                                                                                |          358790 |
+---------------------------------------------------------------------------------------+-----------------+
| Overall_MHQ                                                                           |            2008 |
+---------------------------------------------------------------------------------------+-----------------+
| Age                                                                                   |              12 |
+---------------------------------------------------------------------------------------+-----------------+
| Biological_Sex            

In [15]:
data.to_csv('/Users/jerzybala/Desktop/2003_up_Oct_after_OHE_short.csv', index=False)
